Dados localizaSUS: [https://infoms.saude.gov.br/extensions/DEMAS_C19_Vacina_v2/DEMAS_C19_Vacina_v2.html](https://infoms.saude.gov.br/extensions/DEMAS_C19_Vacina_v2/DEMAS_C19_Vacina_v2.html)

Projeções populacionais municípios: [https://demografiaufrn.net/projecao-populacional/](https://demografiaufrn.net/projecao-populacional/)

In [ ]:
# Importing important libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime
import io
from PIL import Image
import folium
from folium import plugins
import json
from google.colab import drive
from tqdm import tqdm
import scipy.stats as st
import seaborn as sns
import unicodedata
import warnings
warnings.filterwarnings('ignore')
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
!pip install geojson
import csv, json
from geojson import Feature, FeatureCollection, Point

In [ ]:
!pip install geobr
import geobr as gbr

     |████████████████████████████████| 928 kB 8.4 MB/s 
     |████████████████████████████████| 6.3 MB 54.9 MB/s 
     |████████████████████████████████| 15.4 MB 33.5 MB/s 


In [ ]:
!pip install geopandas
import geopandas as gpd

In [ ]:
date = '2022-01-30'
date_new = date.replace('-','')

In [ ]:
!wget 'https://github.com/PedroHPCintra/COVID_Brasil/raw/main/ProjMunic-2010_2030.zip'

--2022-02-02 12:19:04--  https://github.com/PedroHPCintra/COVID_Brasil/raw/main/ProjMunic-2010_2030.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/PedroHPCintra/COVID_Brasil/main/ProjMunic-2010_2030.zip [following]
--2022-02-02 12:19:04--  https://raw.githubusercontent.com/PedroHPCintra/COVID_Brasil/main/ProjMunic-2010_2030.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10172707 (9.7M) [application/zip]
Saving to: ‘ProjMunic-2010_2030.zip’

ProjMunic-2010_2030 100%[===================>]   9.70M  --.-KB/s    in 0.1s    

2022-02-02 12:19:05 (88.5 MB/s) - ‘ProjMunic-2010_2030.zip’ saved 

In [ ]:
!unzip 'ProjMunic-2010_2030.zip'

Archive:  ProjMunic-2010_2030.zip
  inflating: ProjMunic-2010_2030.csv  


In [ ]:
age_pyramid = pd.read_csv('ProjMunic-2010_2030.csv')

In [ ]:
age_pyramid_recent = age_pyramid.loc[age_pyramid['Ano'] == 2021].reset_index(drop=True)

In [ ]:
for i in range(len(age_pyramid_recent)):
  age_pyramid_recent['Armenor'][i] = str(age_pyramid_recent['Armenor'][i])[:-1]

age_pyramid_recent['Armenor'] = age_pyramid_recent['Armenor'].astype('string')

In [ ]:
age_pyramid_recent.head()

,Ano,ARmaior,Sexo,Armenor,NomeMunic,0 a 5,5 a 10,10 a 15,15 a 20,20 a 25,25 a 30,30 a 35,35 a 40,40 a 45,45 a 50,50 a 55,55 a 60,60 a 65,65 a 70,70 a 75,75 a 80,80 a 85,85 a 90,90+,Total
0,2021,ACRE,f,120001,Acrelândia - AC,779,776,796,843,828,778,644,654,613,470,420,328,223,179,122,80,40,21,9,8604
1,2021,ACRE,m,120001,Acrelândia - AC,814,807,829,856,870,758,648,636,607,563,465,405,311,241,168,100,53,30,12,9173
2,2021,ACRE,f,120005,Assis Brasil - AC,539,535,527,516,465,417,346,311,268,223,180,131,104,73,44,38,21,10,9,4757
3,2021,ACRE,m,120005,Assis Brasil - AC,559,547,534,480,401,377,337,324,279,226,199,153,130,81,62,37,21,11,6,4764
4,2021,ACRE,f,120010,Brasiléia - AC,980,1012,1055,1176,1149,1051,1014,961,878,747,612,461,346,283,220,155,93,41,27,12262


In [ ]:
df = pd.read_csv('https://github.com/PedroHPCintra/COVID_Brasil/raw/main/Cobertura_municipios/cobertura_vacinal_brasil_por_municipio.csv')

In [ ]:
if df.columns[0] == '[Município]':
  df = df.rename(columns={'[Município]': 'Município'})

In [ ]:
states = ['AC','AL','AM','AP','BA','CE','DF','ES','GO','MA','MG','MS','MT','PA',
          'PB','PE','PI','PR','RJ','RN','RO','RR','RS','SC','SE','SP','TO']
names = ['Acre', 'Alagoas', 'Amazonas', 'Amapá', 'Bahia', 'Ceará', 'Distrito Federal',
         'Espirito Santo', 'Goiás', 'Maranhão', 'Minas Gerias', 'Mato Grosso do Sul',
         'Mato Grosso', 'Pará', 'Paraíba', 'Pernambuco', 'Piauí', 'Paraná', 'Rio de Janeiro',
         'Rio Grande do Norte', 'Rôndonia', 'Roraima', 'Rio Grande do Sul', 'Santa Catarina',
         'Sergipe', 'São Paulo', 'Tocantins']
stds = []
normality = []
norm_p = []
iqrs = []
cis = []
means = []
medians = []

In [ ]:
df.head(5)

,Município,Cód. IBGE,UF,Região,Fabricante,Total de doses aplicadas,1ª Dose D1,2ª Dose D2,Dose Adicional DA,Dose Reforço DR,Dose Única DU
0,Abadia De Goiás,520005,GO,Centro-Oeste,BUTANTAN/SINOVAC,7.293,4.015,3.271,1.0,6,0.0
1,Abadia De Goiás,520005,GO,Centro-Oeste,FIOCRUZ/ASTRAZENECA,7.387,3.792,3.585,3.0,7,0.0
2,Abadia De Goiás,520005,GO,Centro-Oeste,JANSSEN,198,0,0,8.0,24,166.0
3,Abadia De Goiás,520005,GO,Centro-Oeste,PFIZER/BIONTECH,9.839,4.914,3.309,48.0,1.568,0.0
4,Abadia Dos Dourados,310010,MG,Sudeste,BUTANTAN/SINOVAC,2.700,1.394,1.299,1.0,6,0.0


# Homogeneidade

In [ ]:
for j in tqdm(range(len(states))):
  state = states[j]
  df_state = df.loc[df['UF'] == state].reset_index(drop=True)
  df_state = df_state.drop([len(df_state)-1])

  pops = []
  pops_tot = []

  for i in range(len(df_state)):
    df_new = age_pyramid_recent.loc[age_pyramid_recent['Armenor'] == df_state['Cód. IBGE'][i]].reset_index(drop=True)
    pop_10_plus = np.sum(np.sum(df_new)[5:-1])
    pop_all = np.sum(df_new)[-1]
    pops.append(pop_10_plus)
    pops_tot.append(pop_all)

  df_state['Populacao 10+'] = pops
  df_state['Populacao Total'] = pops_tot
  
  for i in range(len(df_state['2ª Dose D2'])):
    try:
      if len(df_state['2ª Dose D2'][i].split('.')) == 2:
        df_state['2ª Dose D2'][i] = int(df_state['2ª Dose D2'][i].split('.')[0]+df_state['2ª Dose D2'][i].split('.')[1])
      elif len(df_state['2ª Dose D2'][i].split('.')) == 3:
        df_state['2ª Dose D2'][i] = int(df_state['2ª Dose D2'][i].split('.')[0]+df_state['2ª Dose D2'][i].split('.')[1]+df_state['2ª Dose D2'][i].split('.')[2])
    except:
      pass
    
    if str(df_state['Dose Única DU'][i]).split('.')[1] == '000' or str(df_state['Dose Única DU'][i]).split('.')[1] == '0':
      df_state['Dose Única DU'][i] = int(str(df_state['Dose Única DU'][i]).split('.')[0])
    elif len(str(df_state['Dose Única DU'][i]).split('.')[1]) > 8:
      df_state['Dose Única DU'][i] = int(str(df_state['Dose Única DU'][i]).split('.')[0]+str(df_state['Dose Única DU'][i]).split('.')[1][:3])
    elif '9999999' in str(df_state['Dose Única DU'][i]).split('.')[1]:
      df_state['Dose Única DU'][i] = int(str(df_state['Dose Única DU'][i]).split('.')[0]+str(df_state['Dose Única DU'][i]).split('.')[1][:3])
    else:
      df_state['Dose Única DU'][i] = int(str(df_state['Dose Única DU'][i]).split('.')[0]+str(df_state['Dose Única DU'][i]).split('.')[1])

  df_state['2ª Dose D2'] = df_state['2ª Dose D2'].astype('float')

  df_pops = pd.read_csv(f'gdrive/MyDrive/Brasil/Populacao_municipios/{state}_info.csv')

  for i in range(len(df_pops)):
    df_pops['Código [-]'][i] = str(df_pops['Código [-]'][i])[:-1]

  df_pops['Código [-]'] = df_pops['Código [-]'].astype('string')

  df_final = {}
  muns = []
  doses = []
  pops = []
  pops_tot = []

  for i in range(len(df_pops)):
    muns.append(df_pops['Município [-]'][i])
    pe_doses = df_state.loc[df_state['Cód. IBGE'] == df_pops['Código [-]'][i]].reset_index(drop=True)
    pops.append(pe_doses['Populacao 10+'][0])
    pops_tot.append(pe_doses['Populacao Total'][0])
    try:
      doses.append(np.sum(pe_doses['2ª Dose D2'].values)+pe_doses.loc[pe_doses['Fabricante'] == 'JANSSEN']['Dose Única DU'].values[0])
    except:
      doses.append(np.sum(pe_doses['2ª Dose D2'].values))

  df_final['Municipio'] = muns
  df_final['Imunização completa'] = np.array(doses)
  df_final['Populacao 10+'] = np.array(pops)
  df_final['Populacao Total'] = np.array(pops_tot)
  df_final['Cobertura vacinal'] = 100*df_final['Imunização completa']/df_final['Populacao 10+']
  df_final['Cobertura vacinal norm'] = df_final['Imunização completa']/df_final['Populacao 10+']

  for i in range(len(df_final['Cobertura vacinal norm'])):
    if df_final['Cobertura vacinal'][i] > 100:
      df_final['Cobertura vacinal'][i] = 100
    if df_final['Cobertura vacinal norm'][i] > 1:
      df_final['Cobertura vacinal norm'][i] = 1

  df_final = pd.DataFrame(df_final)

  fig = plt.figure(figsize=(12,6))
  n,x,_ = plt.hist(df_final['Cobertura vacinal norm'], bins=np.linspace(0,1,41), color = 'lightseagreen', edgecolor='black',
                  alpha = 0.6)
  bin_centers = 0.5*(x[1:]+x[:-1])
  plt.plot(bin_centers,n, color = 'crimson', linestyle='dashed')
  plt.xlabel('Fração completamente imunizada', fontsize = 14)
  plt.ylabel('Quantidade de municipios', fontsize = 14)
  plt.xlim(0,1)
  fig.patch.set_alpha(1)
  for ax in fig.axes:
      ax.patch.set_alpha(1)
  plt.savefig(f'gdrive/MyDrive/COVID/Vaccination/Distributions/Distribution_coverages_{state}.png', dpi = 300, bbox_inches = 'tight')
  plt.close()

  if state == 'MG' or state == 'SC' or state == 'TO':
    !unzip 'gdrive/MyDrive/Brasil/Malha_municipal/{state}_Municipios_2020.zip'
  else:
    !wget 'https://github.com/PedroHPCintra/COVID_Brasil/raw/main/Cobertura_municipios/Shapefiles_municipios/{state}_Municipios_2020.zip'
  !unzip '{state}_Municipios_2020.zip'
  if state == 'SP':
    shp_file = gpd.read_file(f'SP_Municipios_2020/{state}_Municipios_2020.shp')
  else:
    shp_file = gpd.read_file(f'{state}_Municipios_2020.shp')
  shp_file.to_file(f'{state}_Municipios_2020.geojson',
                  driver='GeoJSON')
  pe_mun = f'{state}_Municipios_2020.geojson'
  geo_json_data = json.load(open(pe_mun))

  map = folium.Map(width='100%', height='100%', location = [-15.8420, -47.9428], zoom_start = 6)

  map.choropleth(geo_data = geo_json_data, name = 'Cobertura vacinal completa', data = df_final, columns = ['Municipio', 'Cobertura vacinal norm'],
               key_on = 'feature.properties.NM_MUN', fill_color = 'YlGnBu', fill_opacity = 0.9, line_color = 'black',
               line_opacity = 0.6,
               show = False,
               legend_name = f'Cobertura vacinal completa ({date})',
               bins = [0,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1])
  
  map.save(f'gdrive/MyDrive/COVID/Vaccination/Maps/vaccine_{state}_2_dose.html')

  means.append(np.mean(df_final['Cobertura vacinal']))
  medians.append(np.median(df_final['Cobertura vacinal']))
  stds.append(np.var(df_final['Cobertura vacinal']))
  iqrs.append(st.iqr(df_final['Cobertura vacinal'], rng = (25,75)))
  if state != 'DF':
    cis.append(st.iqr(df_final['Cobertura vacinal'], rng = (2.5,97.5)))
    normality.append(st.shapiro(df_final['Cobertura vacinal'])[0])
    norm_p.append(st.shapiro(df_final['Cobertura vacinal'])[1])
  else:
    cis.append(st.iqr(df_final['Cobertura vacinal'], rng = (2.5,97.5)))
    normality.append(np.nan)
    norm_p.append(np.nan)

  0%|          | 0/27 [00:00<?, ?it/s]

--2022-02-01 04:17:34--  https://github.com/PedroHPCintra/COVID_Brasil/raw/main/Cobertura_municipios/Shapefiles_municipios/AC_Municipios_2020.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/PedroHPCintra/COVID_Brasil/main/Cobertura_municipios/Shapefiles_municipios/AC_Municipios_2020.zip [following]
--2022-02-01 04:17:34--  https://raw.githubusercontent.com/PedroHPCintra/COVID_Brasil/main/Cobertura_municipios/Shapefiles_municipios/AC_Municipios_2020.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1683179 (1.6M) [application/zip]
Saving to: ‘AC_Municipios_2020.zip’

AC_Municipios_2020. 100

  4%|▎         | 1/27 [00:08<03:52,  8.94s/it]

--2022-02-01 04:17:42--  https://github.com/PedroHPCintra/COVID_Brasil/raw/main/Cobertura_municipios/Shapefiles_municipios/AL_Municipios_2020.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/PedroHPCintra/COVID_Brasil/main/Cobertura_municipios/Shapefiles_municipios/AL_Municipios_2020.zip [following]
--2022-02-01 04:17:42--  https://raw.githubusercontent.com/PedroHPCintra/COVID_Brasil/main/Cobertura_municipios/Shapefiles_municipios/AL_Municipios_2020.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1751724 (1.7M) [application/zip]
Saving to: ‘AL_Municipios_2020.zip’

AL_Municipios_2020. 100

  7%|▋         | 2/27 [00:17<03:37,  8.68s/it]

...

 93%|█████████▎| 25/27 [07:41<00:44, 22.10s/it]

--2022-02-01 04:25:34--  https://github.com/PedroHPCintra/COVID_Brasil/raw/main/Cobertura_municipios/Shapefiles_municipios/SP_Municipios_2020.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/PedroHPCintra/COVID_Brasil/main/Cobertura_municipios/Shapefiles_municipios/SP_Municipios_2020.zip [following]
--2022-02-01 04:25:34--  https://raw.githubusercontent.com/PedroHPCintra/COVID_Brasil/main/Cobertura_municipios/Shapefiles_municipios/SP_Municipios_2020.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18734553 (18M) [application/zip]
Saving to: ‘SP_Municipios_2020.zip’

SP_Municipios_2020. 100

 96%|█████████▋| 26/27 [08:23<00:28, 28.07s/it]

Archive:  gdrive/MyDrive/Brasil/Malha_municipal/TO_Municipios_2020.zip
  inflating: TO_Municipios_2020.cpg  
  inflating: TO_Municipios_2020.dbf  
  inflating: TO_Municipios_2020.prj  
  inflating: TO_Municipios_2020.shp  
  inflating: TO_Municipios_2020.shx  
unzip:  cannot find or open TO_Municipios_2020.zip, TO_Municipios_2020.zip.zip or TO_Municipios_2020.zip.ZIP.


100%|██████████| 27/27 [09:09<00:00, 20.34s/it]


In [ ]:
br_final = {}

br_final['Estados'] = names
br_final['Siglas'] = states
br_final['Médias'] = means
br_final['Medianas'] = medians
br_final['Variância'] = stds
br_final['SD'] = np.sqrt(np.array(stds))
br_final['Normality'] = normality
br_final['p-value'] = norm_p
br_final['IQR'] = iqrs
br_final['CIs'] = np.array(cis)/100
br_final['Homogeneidade'] = 1 - np.array(cis)/100

In [ ]:
br_final = pd.DataFrame(br_final)

In [ ]:
br_final

,Estados,Siglas,Médias,Medianas,Variância,SD,Normality,p-value,IQR,CIs,Homogeneidade
0,Acre,AC,48.499279,45.677237,116.822642,10.808452,0.947032,2.755004e-01,11.420765,0.364730,0.635270
1,Alagoas,AL,66.691788,67.263912,158.551955,12.591742,0.987557,4.615096e-01,17.215226,0.472044,0.527956
2,Amazonas,AM,43.137781,39.614761,229.070859,15.135087,0.962838,5.760743e-02,24.919866,0.530938,0.469062
3,Amapá,AP,38.607311,37.656413,87.724976,9.366161,0.942993,3.873012e-01,13.790115,0.315051,0.684949
4,Bahia,BA,67.162319,68.531469,271.380161,16.473620,0.981805,4.245052e-05,20.164477,0.696535,0.303465
5,Ceará,CE,64.923746,66.137851,279.769256,16.726304,0.985121,4.836874e-02,22.328723,0.709892,0.290108
6,Distrito Federal,DF,44.839107,44.839107,0.000000,0.000000,NaN,NaN,0.000000,0.000000,1.000000
7,Espirito Santo,ES,77.799865,77.590960,93.015417,9.644450,0.990480,8.355545e-01,14.406124,0.341399,0.658601
8,Goiás,GO,76.586080,77.567688,210.428878,14.506167,0.971391,7.393352e-05,19.887049,0.575683,0.424317
9,Maranhão,MA,51.617957,51.383255,248.055597,15.749781,0.986109,3.230777e-02,20.944229,0.627256,0.372744


In [ ]:
st.spearmanr(br_final['SD'], br_final['CIs'])

SpearmanrResult(correlation=0.9572649572649573, pvalue=5.579061557313431e-15)

In [ ]:
max(br_final['Homogeneidade'])

1.0

In [ ]:
!unzip 'gdrive/MyDrive/Brasil/Malha_municipal/BR_UF_2020.zip'

Archive:  gdrive/MyDrive/Brasil/Malha_municipal/BR_UF_2020.zip
  inflating: BR_UF_2020.cpg          
  inflating: BR_UF_2020.dbf          
  inflating: BR_UF_2020.prj          
  inflating: BR_UF_2020.shp          
  inflating: BR_UF_2020.shx          


In [ ]:
shp_file = gpd.read_file('BR_UF_2020.shp')

In [ ]:
shp_file.to_file('BR_UF_2020.geojson',
                 driver='GeoJSON')

In [ ]:
br_states = 'BR_UF_2020.geojson'

In [ ]:
geo_json_data = json.load(open(br_states))

In [ ]:
geo_json_data['features'][0]['properties']

{'CD_UF': '11', 'NM_REGIAO': 'Norte', 'NM_UF': 'Rondônia', 'SIGLA_UF': 'RO'}

In [ ]:
map = folium.Map(width='100%', height='100%', location = [-15.8420, -47.9428], zoom_start = 5)

In [ ]:
map.choropleth(geo_data = geo_json_data, name = 'Homogeneidade na cobertura vacinal', data = br_final,
               columns = ['Siglas', 'Homogeneidade'],
               key_on = 'feature.properties.SIGLA_UF', fill_color = 'RdYlBu', fill_opacity = 0.9, line_color = 'black',
               line_opacity = 0.6,
               show = False,
               legend_name = f'Homogeneidade na cobertura vacinal ({date})',
               bins = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1])

In [ ]:
map.save('br_homogeneity.html')

# Mapa mesorregiões

In [ ]:
pops = []
pops_old = []
pops_tot = []

for i in range(len(df)):
  df_new = age_pyramid_recent.loc[age_pyramid_recent['Armenor'] == df['Cód. IBGE'][i]].reset_index(drop=True)
  pop_10_plus = np.sum(np.sum(df_new)[5:-1])
  pop_25_plus = np.sum(np.sum(df_new)[9:-1])
  pop_all = np.sum(df_new)[-1]
  pops.append(pop_10_plus)
  pops_tot.append(pop_all)
  pops_old.append(pop_25_plus)

df['Populacao 10+'] = pops
df['Populacao 25+'] = pops_old
df['Populacao Total'] = pops_tot

In [ ]:
df.head()

,Município,Cód. IBGE,UF,Região,Fabricante,Total de doses aplicadas,1ª Dose D1,2ª Dose D2,Dose Adicional DA,Dose Reforço DR,Dose Única DU,Populacao 10+,População 25+,Populacao Total,Populacao 25+
0,abadia de goias,520005,GO,Centro-Oeste,BUTANTAN/SINOVAC,7.293,4.015,3271.0,1.0,6.0,0.0,8363.0,5654.0,9068.0,5654.0
1,abadia de goias,520005,GO,Centro-Oeste,FIOCRUZ/ASTRAZENECA,7.387,3.792,3585.0,3.0,7.0,0.0,8363.0,5654.0,9068.0,5654.0
2,abadia de goias,520005,GO,Centro-Oeste,JANSSEN,198,0,0.0,8.0,24.0,166.0,8363.0,5654.0,9068.0,5654.0
3,abadia de goias,520005,GO,Centro-Oeste,PFIZER/BIONTECH,9.839,4.914,3309.0,48.0,1568.0,0.0,8363.0,5654.0,9068.0,5654.0
4,abadia dos dourados,310010,MG,Sudeste,BUTANTAN/SINOVAC,2.700,1.394,1299.0,1.0,6.0,0.0,6601.0,4922.0,6975.0,4922.0


In [ ]:
for i in range(len(df['2ª Dose D2'])):
  try:
    if len(df['2ª Dose D2'][i].split('.')) == 2:
      df['2ª Dose D2'][i] = int(df['2ª Dose D2'][i].split('.')[0]+df['2ª Dose D2'][i].split('.')[1])
    elif len(df['2ª Dose D2'][i].split('.')) == 3:
      df['2ª Dose D2'][i] = int(df['2ª Dose D2'][i].split('.')[0]+df['2ª Dose D2'][i].split('.')[1]+df['2ª Dose D2'][i].split('.')[2])
  except:
    pass
  
  if str(df['Dose Única DU'][i]).split('.')[1] == '000' or str(df['Dose Única DU'][i]).split('.')[1] == '0':
    df['Dose Única DU'][i] = int(str(df['Dose Única DU'][i]).split('.')[0])
  elif len(str(df['Dose Única DU'][i]).split('.')[1]) > 8:
    df['Dose Única DU'][i] = int(str(df['Dose Única DU'][i]).split('.')[0]+str(df['Dose Única DU'][i]).split('.')[1][:3])
  elif len(str(df['Dose Única DU'][i]).split('.')) == 2:
    df['Dose Única DU'][i] = int(str(df['Dose Única DU'][i]).split('.')[0]+str(df['Dose Única DU'][i]).split('.')[1])
  else:
    print(df['Dose Única DU'][i])

for i in range(len(df['Dose Reforço DR'])):
  try:
    if len(df['Dose Reforço DR'][i].split('.')) == 2:
      df['Dose Reforço DR'][i] = int(df['Dose Reforço DR'][i].split('.')[0]+df['Dose Reforço DR'][i].split('.')[1])
    elif len(df['Dose Reforço DR'][i].split('.')) == 3:
      df['Dose Reforço DR'][i] = int(df['Dose Reforço DR'][i].split('.')[0]+df['Dose Reforço DR'][i].split('.')[1]+df['Dose Reforço DR'][i].split('.')[2])
  except:
    pass
  
  if str(df['Dose Adicional DA'][i]).split('.')[1] == '000' or str(df['Dose Adicional DA'][i]).split('.')[1] == '0':
    df['Dose Adicional DA'][i] = int(str(df['Dose Adicional DA'][i]).split('.')[0])
  elif len(str(df['Dose Adicional DA'][i]).split('.')[1]) > 8:
    df['Dose Adicional DA'][i] = int(str(df['Dose Adicional DA'][i]).split('.')[0]+str(df['Dose Adicional DA'][i]).split('.')[1][:3])
  elif len(str(df['Dose Adicional DA'][i]).split('.')) == 2:
    df['Dose Adicional DA'][i] = int(str(df['Dose Adicional DA'][i]).split('.')[0]+str(df['Dose Adicional DA'][i]).split('.')[1])
  else:
    print(df['Dose Adicional DA'][i])

df['Dose Reforço DR'] = df['Dose Reforço DR'].astype('float')
df['2ª Dose D2'] = df['2ª Dose D2'].astype('float')

In [ ]:
meso = pd.read_csv('gdrive/MyDrive/Brasil/Mesorregioes_e_municipios.csv')

In [ ]:
br_final = {}

mesoregioes = []
for i in range(len(meso)):
  if i == 0:
    mesoregiao = meso['Mesorregiao'][i]
    mesoregioes.append(mesoregiao)
  else:
    mesoregiao = meso['Mesorregiao'][i]
    if mesoregiao != mesoregioes[-1]:
      mesoregioes.append(mesoregiao)

In [ ]:
def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

In [ ]:
for i in range(len(meso)):
  meso['Municipios'][i] = remove_accents(meso['Municipios'][i].lower())
  if '-' in meso['Municipios'][i]:
    if len(meso['Municipios'][i].split('-')) == 2:
      meso['Municipios'][i] = meso['Municipios'][i].split('-')[0] + ' ' + meso['Municipios'][i].split('-')[1]
    elif len(meso['Municipios'][i].split('-')) == 3:
      meso['Municipios'][i] = meso['Municipios'][i].split('-')[0] + ' ' + meso['Municipios'][i].split('-')[1] + ' ' + meso['Municipios'][i].split('-')[2]
    else:
      print(meso['Municipios'][i])

In [ ]:
for i in range(len(df)):
  df['Município'][i] = remove_accents(df['Município'][i].lower())
  if '-' in df['Município'][i]:
    if len(df['Município'][i].split('-')) == 2:
      df['Município'][i] = df['Município'][i].split('-')[0] + ' ' + df['Município'][i].split('-')[1]
    elif len(df['Município'][i].split('-')) == 3:
      df['Município'][i] = df['Município'][i].split('-')[0] + ' ' + df['Município'][i].split('-')[1] + ' ' + df['Município'][i].split('-')[2]
    else:
      print(df['Município'][i])

In [ ]:
discrep = 0
for i in range(len(mesoregioes)):
  df_new = meso.loc[meso['Mesorregiao'] == mesoregioes[i]].reset_index(drop=True)
  for j in range(len(df_new)):
    if df_new['Municipios'][j] == 'itapaje':
      mun = df.loc[df['Município'] == 'itapage'].reset_index(drop=True)
    elif df_new['Municipios'][j] == 'sao vicente do serido':
      mun = df.loc[df['Município'] == 'serido'].reset_index(drop=True)
    elif df_new['Municipios'][j] == 'amparo do sao francisco':
      mun = df.loc[df['Município'] == 'amparo de sao francisco'].reset_index(drop=True)
    elif df_new['Municipios'][j] == 'tabocao':
      mun = df.loc[df['Município'] == 'fortaleza do tabocao'].reset_index(drop=True)
    elif df_new['Municipios'][j] == 'sao tome das letras':
      mun = df.loc[df['Município'] == 'sao thome das letras'].reset_index(drop=True)
    elif df_new['Municipios'][j] == 'amparo do serra':
      mun = df.loc[df['Município'] == 'amparo da serra'].reset_index(drop=True)
    elif df_new['Municipios'][j] == 'embu das artes':
      mun = df.loc[df['Município'] == 'embu'].reset_index(drop=True)
    else:
      mun = df.loc[df['Município'] == df_new['Municipios'][j]].reset_index(drop=True)
    if len(mun['1ª Dose D1']) < 1:
      print(df_new['Municipios'][j])
      discrep += 1

In [ ]:
discrep

0

In [ ]:
final_populacoes = []
final_populacoes_25 = []
final_doses = []
third_doses = []

discrep = 0
for i in range(len(mesoregioes)):
  populacoes = []
  populacoes_25 = []
  doses = []
  doses_3 = []
  df_new = meso.loc[meso['Mesorregiao'] == mesoregioes[i]].reset_index(drop=True)
  for j in range(len(df_new)):
    if df_new['Municipios'][j] == 'itapaje':
      mun = df.loc[df['Município'] == 'itapage'].reset_index(drop=True)
    elif df_new['Municipios'][j] == 'sao vicente do serido':
      mun = df.loc[df['Município'] == 'serido'].reset_index(drop=True)
    elif df_new['Municipios'][j] == 'amparo do sao francisco':
      mun = df.loc[df['Município'] == 'amparo de sao francisco'].reset_index(drop=True)
    elif df_new['Municipios'][j] == 'tabocao':
      mun = df.loc[df['Município'] == 'fortaleza do tabocao'].reset_index(drop=True)
    elif df_new['Municipios'][j] == 'sao tome das letras':
      mun = df.loc[df['Município'] == 'sao thome das letras'].reset_index(drop=True)
    elif df_new['Municipios'][j] == 'amparo do serra':
      mun = df.loc[df['Município'] == 'amparo da serra'].reset_index(drop=True)
    elif df_new['Municipios'][j] == 'embu das artes':
      mun = df.loc[df['Município'] == 'embu'].reset_index(drop=True)
    else:
      mun = df.loc[df['Município'] == df_new['Municipios'][j]].reset_index(drop=True)
    if len(mun['1ª Dose D1']) < 1:
      print(df_new['Municipios'][j])
      discrep += 1
    populacoes.append(mun['Populacao 10+'].values[0])
    populacoes_25.append(mun['Populacao 25+'].values[0])
    try:
      all = np.sum(mun['2ª Dose D2'].values) + mun.loc[mun['Fabricante'] == 'JANSSEN']['Dose Única DU'].values[0]
      third = np.sum(mun['Dose Adicional DA'].values) + np.sum(mun['Dose Reforço DR'].values)
      doses_3.append(third)
      if all > 1e10:
        print(mun.loc[mun['Fabricante'] == 'JANSSEN']['Dose Única DU'].values)
      doses.append(all)
    except:
      doses.append(np.sum(mun['2ª Dose D2'].values))
  populacoes = np.array(populacoes)
  populacoes_25 = np.array(populacoes_25)
  doses = np.array(doses)
  final_populacoes.append(np.sum(populacoes))
  final_populacoes_25.append(np.sum(populacoes_25))
  final_doses.append(sum(doses))
  third_doses.append(sum(doses_3))

In [ ]:
br_final['Mesorregiao'] = np.array(mesoregioes)
br_final['População 10+'] = np.array(final_populacoes)
br_final['População 25+'] = np.array(final_populacoes_25)
br_final['Imunização completa'] = np.array(final_doses)
br_final['Terceira dose'] = np.array(third_doses)
br_final['Fração vacinada'] = br_final['Imunização completa']/br_final['População 10+']
br_final['Fração reforçada'] = br_final['Terceira dose']/br_final['População 10+']
br_final['Elegível reforçado'] = br_final['Terceira dose']/br_final['População 25+']

In [ ]:
for i in range(len(br_final['Fração vacinada'])):
  if br_final['Fração vacinada'][i] > 1:
    br_final['Fração vacinada'][i] = 1

for i in range(len(br_final['Fração reforçada'])):
  if br_final['Fração reforçada'][i] > 1:
    br_final['Fração reforçada'][i] = 1

for i in range(len(br_final['Elegível reforçado'])):
  if br_final['Elegível reforçado'][i] > 1:
    br_final['Elegível reforçado'][i] = 1

br_final = pd.DataFrame(br_final)

In [ ]:
br_final

,Mesorregiao,População 10+,População 25+,Imunização completa,Terceira dose,Fração vacinada,Fração reforçada,Elegível reforçado
0,Noroeste Goiano,237077.0,164492.0,207108.0,52820.0,0.873590,0.222797,0.321110
1,Norte Goiano,289767.0,199817.0,217150.0,44595.0,0.749395,0.153900,0.223179
2,Centro Goiano,3382186.0,2342161.0,2424580.0,618052.0,0.716868,0.182737,0.263881
3,Leste Goiano,1338834.0,855301.0,741167.0,139681.0,0.553591,0.104330,0.163312
4,Sul Goiano,1486056.0,1037510.0,1134869.0,274675.0,0.763678,0.184835,0.264744
...,...,...,...,...,...,...,...,...
132,Serrana,407264.0,288004.0,343074.0,78124.0,0.842387,0.191826,0.271260
133,Vale do Itajaí,1860920.0,1320803.0,1270235.0,225902.0,0.682584,0.121393,0.171034
134,Grande Florianópolis,1127180.0,827762.0,904020.0,163878.0,0.802019,0.145388,0.197977
135,Sul Catarinense,1300778.0,935646.0,974074.0,269185.0,0.748840,0.206942,0.287700


In [ ]:
min(br_final['Fração vacinada'])

0.33847641719982147

In [ ]:
min(br_final['Elegível reforçado'])

0.07691685756636572

In [ ]:
br_final.loc[br_final['Fração reforçada'] == max(br_final['Fração reforçada'])]

,Mesorregiao,População 10+,População 25+,Imunização completa,Terceira dose,Fração vacinada,Fração reforçada,Elegível reforçado
37,Borborema,380750.0,248249.0,1139760.0,371004.0,1.0,0.974403,1.0


In [ ]:
!unzip 'gdrive/MyDrive/Brasil/Malha_municipal/BR_Mesorregioes_2020.zip'
shp_file = gpd.read_file(f'BR_Mesorregioes_2020.shp')
shp_file.to_file(f'BR_Mesorregioes_2020.geojson',
               driver='GeoJSON')
pe_mun = f'BR_Mesorregioes_2020.geojson'
geo_json_data = json.load(open(pe_mun))

Archive:  gdrive/MyDrive/Brasil/Malha_municipal/BR_Mesorregioes_2020.zip
replace BR_Mesorregioes_2020.cpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: BR_Mesorregioes_2020.cpg  
  inflating: BR_Mesorregioes_2020.dbf  
  inflating: BR_Mesorregioes_2020.prj  
  inflating: BR_Mesorregioes_2020.shp  
  inflating: BR_Mesorregioes_2020.shx  


In [ ]:
for i in range(len(geo_json_data['features'])):
  if geo_json_data['features'][i]['properties']['SIGLA_UF'] == 'DF':
    print(geo_json_data['features'][i]['properties'])

{'CD_MESO': '5301', 'NM_MESO': 'Distrito Federal', 'SIGLA_UF': 'DF'}


In [ ]:
color_2nd = 'YlGnBu'
color_2nd_colorblind = 'YlGn'

In [ ]:
map = folium.Map(width='100%', height='100%', location = [-15.8420, -47.9428], zoom_start = 4)
map_cb = folium.Map(width='100%', height='100%', location = [-15.8420, -47.9428], zoom_start = 4)

map.choropleth(geo_data = geo_json_data, name = 'Cobertura vacinal 2 doses/dose única', data = br_final, columns = ['Mesorregiao', 'Fração vacinada'],
              key_on = 'feature.properties.NM_MESO', fill_color = color_2nd, fill_opacity = 0.9, line_color = 'black',
              line_opacity = 0.4,
              show = False,
              legend_name = f'Cobertura vacinal 2 doses/dose única ({date})',
              bins = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1])

map_cb.choropleth(geo_data = geo_json_data, name = 'Cobertura vacinal 2 doses/dose única', data = br_final, columns = ['Mesorregiao', 'Fração vacinada'],
              key_on = 'feature.properties.NM_MESO', fill_color = color_2nd_colorblind, fill_opacity = 0.9, line_color = 'black',
              line_opacity = 0.4,
              show = False,
              legend_name = f'Cobertura vacinal 2 doses/dose única ({date})',
              bins = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1])

map.save('vaccine_BR_2_dose.html')
map_cb.save('vaccine_BR_2_dose_colorblind.html')

In [ ]:
color_3rd = 'RdPu'

map = folium.Map(width='100%', height='100%', location = [-15.8420, -47.9428], zoom_start = 4)

map.choropleth(geo_data = geo_json_data, name = 'Cobertura vacinal 3 doses/reforço', data = br_final, columns = ['Mesorregiao', 'Elegível reforçado'],
              key_on = 'feature.properties.NM_MESO', fill_color = color_3rd, fill_opacity = 0.9, line_color = 'black',
              line_opacity = 0.4,
              show = False,
              legend_name = f'Cobertura vacinal 3 doses/reforço ({date})',
              bins = [0,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1])

map.save('vaccine_BR_3_dose.html')

# Mapa regiões de saúde

In [ ]:
health_reg = gbr.read_health_region(year=2013, macro = False)

In [ ]:
health_reg.head()

,code_health_region,name_health_region,code_state,abbrev_state,name_state,geometry
0,11001,Vale do Jamari,11.0,RO,Rondônia,"POLYGON ((-62.42318 -9.80583, -62.44172 -9.818..."
1,11002,Café,11.0,RO,Rondônia,"POLYGON ((-61.00060 -10.99224, -61.00060 -10.9..."
2,11003,Central,11.0,RO,Rondônia,"POLYGON ((-62.67078 -10.28778, -62.66994 -10.2..."
3,11004,Madeira-Mamoré,11.0,RO,Rondônia,"POLYGON ((-64.20502 -11.91591, -64.21394 -11.9..."
4,11005,Zona da Mata,11.0,RO,Rondônia,"POLYGON ((-62.41771 -13.11894, -62.43446 -13.1..."


In [ ]:
!wget 'https://github.com/PedroHPCintra/COVID_Brasil/raw/main/Cobertura_municipios/Composicao_regioes_de_saude.zip'
!unzip 'Composicao_regioes_de_saude.zip'

--2022-02-02 12:19:19--  https://github.com/PedroHPCintra/COVID_Brasil/raw/main/Cobertura_municipios/Composicao_regioes_de_saude.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/PedroHPCintra/COVID_Brasil/main/Cobertura_municipios/Composicao_regioes_de_saude.zip [following]
--2022-02-02 12:19:20--  https://raw.githubusercontent.com/PedroHPCintra/COVID_Brasil/main/Cobertura_municipios/Composicao_regioes_de_saude.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84151 (82K) [application/zip]
Saving to: ‘Composicao_regioes_de_saude.zip’

Composicao_regioes_ 100%[===================>]  82.18K  

In [ ]:
health_composition = pd.read_csv('Composicao_regioes_de_saude.csv')

In [ ]:
health_composition['co_regiao_saude'] = health_composition['co_regiao_saude'].astype(int)
health_composition['co_regiao_saude'] = health_composition['co_regiao_saude'].astype(str)
health_composition['ibge'] = health_composition['ibge'].astype(str)

In [ ]:
health_composition.head()

,ibge,no_cidade,area_territorial,co_regiao_saude
0,430750,Espumoso,783069.0,43019
1,312740,Gonçalves,187353.0,31003
2,240450,Guamaré,258962.0,24003
3,351750,Guapiaçu,326.0,35155
4,312530,Faria Lemos,165224.0,31042


In [ ]:
pops = []
pops_old = []
pops_tot = []

for i in range(len(df)):
  df_new = age_pyramid_recent.loc[age_pyramid_recent['Armenor'] == df['Cód. IBGE'][i]].reset_index(drop=True)
  pop_10_plus = np.sum(np.sum(df_new)[5:-1])
  pop_25_plus = np.sum(np.sum(df_new)[9:-1])
  pop_all = np.sum(df_new)[-1]
  pops.append(pop_10_plus)
  pops_tot.append(pop_all)
  pops_old.append(pop_25_plus)

df['Populacao 10+'] = pops
df['Populacao 25+'] = pops_old
df['Populacao Total'] = pops_tot

In [ ]:
for i in range(len(df['2ª Dose D2'])):
  try:
    if len(df['2ª Dose D2'][i].split('.')) == 2:
      df['2ª Dose D2'][i] = int(df['2ª Dose D2'][i].split('.')[0]+df['2ª Dose D2'][i].split('.')[1])
    elif len(df['2ª Dose D2'][i].split('.')) == 3:
      df['2ª Dose D2'][i] = int(df['2ª Dose D2'][i].split('.')[0]+df['2ª Dose D2'][i].split('.')[1]+df['2ª Dose D2'][i].split('.')[2])
  except:
    pass
  
  if str(df['Dose Única DU'][i]).split('.')[1] == '000' or str(df['Dose Única DU'][i]).split('.')[1] == '0':
    df['Dose Única DU'][i] = int(str(df['Dose Única DU'][i]).split('.')[0])
  elif len(str(df['Dose Única DU'][i]).split('.')[1]) > 8:
    df['Dose Única DU'][i] = int(str(df['Dose Única DU'][i]).split('.')[0]+str(df['Dose Única DU'][i]).split('.')[1][:3])
  elif len(str(df['Dose Única DU'][i]).split('.')) == 2:
    df['Dose Única DU'][i] = int(str(df['Dose Única DU'][i]).split('.')[0]+str(df['Dose Única DU'][i]).split('.')[1])
  else:
    print(df['Dose Única DU'][i])

for i in range(len(df['Dose Reforço DR'])):
  try:
    if len(df['Dose Reforço DR'][i].split('.')) == 2:
      df['Dose Reforço DR'][i] = int(df['Dose Reforço DR'][i].split('.')[0]+df['Dose Reforço DR'][i].split('.')[1])
    elif len(df['Dose Reforço DR'][i].split('.')) == 3:
      df['Dose Reforço DR'][i] = int(df['Dose Reforço DR'][i].split('.')[0]+df['Dose Reforço DR'][i].split('.')[1]+df['Dose Reforço DR'][i].split('.')[2])
  except:
    pass
  
  if str(df['Dose Adicional DA'][i]).split('.')[1] == '000' or str(df['Dose Adicional DA'][i]).split('.')[1] == '0':
    df['Dose Adicional DA'][i] = int(str(df['Dose Adicional DA'][i]).split('.')[0])
  elif len(str(df['Dose Adicional DA'][i]).split('.')[1]) > 8:
    df['Dose Adicional DA'][i] = int(str(df['Dose Adicional DA'][i]).split('.')[0]+str(df['Dose Adicional DA'][i]).split('.')[1][:3])
  elif len(str(df['Dose Adicional DA'][i]).split('.')) == 2:
    df['Dose Adicional DA'][i] = int(str(df['Dose Adicional DA'][i]).split('.')[0]+str(df['Dose Adicional DA'][i]).split('.')[1])
  else:
    print(df['Dose Adicional DA'][i])

df['Dose Reforço DR'] = df['Dose Reforço DR'].astype('float')
df['2ª Dose D2'] = df['2ª Dose D2'].astype('float')

In [ ]:
br_final = {}

regioes_saude = []
for i in range(len(health_reg)):
  if i == 0:
    regiao = health_reg['code_health_region'][i]
    regioes_saude.append(regiao)
  else:
    regiao = health_reg['code_health_region'][i]
    if regiao != regioes_saude[-1]:
      regioes_saude.append(regiao)

In [ ]:
final_populacoes = []
final_populacoes_25 = []
final_doses = []
third_doses = []

discrep = 0
for i in range(len(regioes_saude)):
  populacoes = []
  populacoes_25 = []
  doses = []
  doses_3 = []
  df_new = health_composition.loc[health_composition['co_regiao_saude'] == regioes_saude[i]].reset_index(drop=True)
  for j in range(len(df_new)):
    mun = df.loc[df['Cód. IBGE'] == df_new['ibge'][j]].reset_index(drop=True)
    if len(mun['1ª Dose D1']) < 1:
      print(df_new['Municipios'][j])
      discrep += 1
    populacoes.append(mun['Populacao 10+'].values[0])
    populacoes_25.append(mun['Populacao 25+'].values[0])
    try:
      all = np.sum(mun['2ª Dose D2'].values) + mun.loc[mun['Fabricante'] == 'JANSSEN']['Dose Única DU'].values[0]
      third = np.sum(mun['Dose Adicional DA'].values) + np.sum(mun['Dose Reforço DR'].values)
      doses_3.append(third)
      if all > 1e10:
        print(mun.loc[mun['Fabricante'] == 'JANSSEN']['Dose Única DU'].values)
      doses.append(all)
    except:
      doses.append(np.sum(mun['2ª Dose D2'].values))
  populacoes = np.array(populacoes)
  populacoes_25 = np.array(populacoes_25)
  doses = np.array(doses)
  final_populacoes.append(np.sum(populacoes))
  final_populacoes_25.append(np.sum(populacoes_25))
  final_doses.append(sum(doses))
  third_doses.append(sum(doses_3))

In [ ]:
br_final['Regiao saude'] = np.array(regioes_saude)
br_final['População 10+'] = np.array(final_populacoes)
br_final['População 25+'] = np.array(final_populacoes_25)
br_final['Imunização completa'] = np.array(final_doses)
br_final['Terceira dose'] = np.array(third_doses)
br_final['Fração vacinada'] = br_final['Imunização completa']/br_final['População 10+']
br_final['Fração reforçada'] = br_final['Terceira dose']/br_final['População 10+']
br_final['Elegível reforçado'] = br_final['Terceira dose']/br_final['População 25+']

In [ ]:
for i in range(len(br_final['Fração vacinada'])):
  if br_final['Fração vacinada'][i] > 1:
    br_final['Fração vacinada'][i] = 1

for i in range(len(br_final['Fração reforçada'])):
  if br_final['Fração reforçada'][i] > 1:
    br_final['Fração reforçada'][i] = 1

for i in range(len(br_final['Elegível reforçado'])):
  if br_final['Elegível reforçado'][i] > 1:
    br_final['Elegível reforçado'][i] = 1

br_final = pd.DataFrame(br_final)

In [ ]:
br_final

,Regiao saude,População 10+,População 25+,Imunização completa,Terceira dose,Fração vacinada,Fração reforçada,Elegível reforçado
0,11001,267058.0,168178.0,141478.0,21630.0,0.529765,0.080994,0.128614
1,11002,155500.0,104703.0,114599.0,24137.0,0.736971,0.155222,0.230528
2,11003,305552.0,206671.0,221724.0,50871.0,0.725651,0.166489,0.246145
3,11004,618114.0,401267.0,367986.0,84803.0,0.595337,0.137196,0.211338
4,11005,125515.0,85327.0,88976.0,18626.0,0.708887,0.148397,0.218290
...,...,...,...,...,...,...,...,...
433,52015,465910.0,319189.0,291177.0,64696.0,0.624964,0.138859,0.202689
434,52016,221849.0,153544.0,152835.0,33256.0,0.688915,0.149904,0.216589
435,52017,234238.0,168636.0,190801.0,42554.0,0.814560,0.181670,0.252342
436,52018,171932.0,116717.0,113276.0,24201.0,0.658842,0.140759,0.207348


In [ ]:
color_2nd = 'YlGnBu'
color_2nd_colorblind = 'YlGn'

In [ ]:
map = folium.Map(width='100%', height='100%', location = [-15.8420, -47.9428], zoom_start = 4)
map_cb = folium.Map(width='100%', height='100%', location = [-15.8420, -47.9428], zoom_start = 4)

map.choropleth(geo_data = health_reg.to_json(), name = 'Cobertura vacinal 2 doses/dose única', data = br_final, columns = ['Regiao saude', 'Fração vacinada'],
              key_on = 'feature.properties.code_health_region', fill_color = color_2nd, fill_opacity = 0.9, line_color = 'black',
              line_opacity = 0.4,
              show = False,
              legend_name = f'Cobertura vacinal 2 doses/dose única ({date})',
              bins = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1])

map_cb.choropleth(geo_data = health_reg.to_json(), name = 'Cobertura vacinal 2 doses/dose única', data = br_final, columns = ['Regiao saude', 'Fração vacinada'],
              key_on = 'feature.properties.code_health_region', fill_color = color_2nd_colorblind, fill_opacity = 0.9, line_color = 'black',
              line_opacity = 0.4,
              show = False,
              legend_name = f'Cobertura vacinal 2 doses/dose única ({date})',
              bins = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1])

map.save('vaccine_BR_2_dose_health_region.html')
map_cb.save('vaccine_BR_2_dose_health_region_colorblind.html')

In [ ]:
color_3rd = 'RdPu'

map = folium.Map(width='100%', height='100%', location = [-15.8420, -47.9428], zoom_start = 4)

map.choropleth(geo_data = health_reg.to_json(), name = 'Cobertura vacinal 3 doses/reforço', data = br_final, columns = ['Regiao saude', 'Elegível reforçado'],
              key_on = 'feature.properties.code_health_region', fill_color = color_3rd, fill_opacity = 0.9, line_color = 'black',
              line_opacity = 0.4,
              show = False,
              legend_name = f'Cobertura vacinal 3 doses/reforço ({date})',
              bins = [0,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1])

map.save('vaccine_BR_3_dose_health_region.html')